# NanoSolveIT KB and eNanoMapper Ontology
This notebook generates some visualizations about the usage of eNanoMapper ontology terms described to use either a nanomaterial or measurement variable in the NanoSolveIT Knowledge Base.

# Imports

In [ ]:
import numpy as np
import pandas as pd
import re
import plotly.express as px # remove
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as pyo
import preprocess as prep
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Data preprocessing
The data received from the partners is a result of the following query against the NanoSolveIT Knowledge Base:
> Which eNM-ontology terms are used in the NanoSolveIT KB to describe either a Nanomaterial or any Measurement Variable

The SPARQL query used for this purpose did not render the data in the shape that would be most useful for these visualizations, so a bit of data preprocessing was needed. The steps are included in ta separate script for reference.

In [ ]:
data = prep.do_preprocess()

# Visualization
## Sunburst chart
The following charts use the [Plotly Sunburst chart](https://plotly.com/python/sunburst-charts/) to visualize how the NanoSolveIT KB is represented in the eNanoMapper ontology in terms of nanomaterial or variable description, presence in datasets, and links to OECD guidelines.

Click on any ring segment to expand

In [ ]:


# Preprocess the data to obtain the columns needed for sunburst charting
from code import interact
from turtle import width
from unittest import makeSuite


datasb = pd.DataFrame(columns = ["eNanoMapper terms", "type", "iri", "label", "Number of observations"])
vars = ["nanomaterial", "variable", "oecd_guideline", "dataset"]
for var in vars:
    df = data[data[var]!=""][["iri", "label", var]].rename(columns={var:"target"})
    df["Number of observations"] = [1 for i in range(len(df["iri"]))]
    df["eNanoMapper terms"] = ["eNanoMapper terms" for i in range(len(df["iri"]))]
    df["type"] = [var for i in range(len(df["iri"]))]
    datasb = pd.concat([datasb, df], axis=0)

def make_sb(datasb, field):
    if field !="all":
        figdata = datasb[datasb["type"] == field]  
        fig = px.sunburst(figdata,
                    path = ["type", "iri", "label", "target"], values = "Number of observations",
                    width=1200, height=1200)
    else:
        figdata = datasb
        fig = px.sunburst(figdata,
                    path = ["eNanoMapper terms", "type", "iri", "label", "target"], values = "Number of observations",
                    width=1000, height=1000)
    return fig


nm = make_sb(datasb, "nanomaterial")
ds = make_sb(datasb, "dataset")
va = make_sb(datasb, "variable")
og = make_sb(datasb, "oecd_guideline")
fig1 = make_subplots(rows=1, cols=4, specs=[
    [{"type": "sunburst"}, {"type": "sunburst"},
    {"type": "sunburst"}, {"type": "sunburst"}]],)

fig1.add_trace(nm.data[0], row=1, col=1)
fig1.add_trace(ds.data[0], row=1,col=2)
fig1.add_trace(va.data[0], row=1,col=3)
fig1.add_trace(og.data[0], row=1,col=4)
fig1.show()

